[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/antoniovfonseca/agentic-ai-global-lulc/blob/main/notebooks/overview-glance.ipynb)

## **1. Import of geospatial processing and visualization modules.**
---



In [ ]:
# Filter out FutureWarning messages to keep the output clean
import warnings
warnings.simplefilter(
    action='ignore',
    category=FutureWarning
)

# Import the Earth Engine API
import ee
from utils import get_glance_map

If you are running this notebook in Google Colab, please uncomment and run the following cell to install the required packages.

## **2. Earth Engine session authentication and initialization.**


---
See the Google instructions [here](https://developers.google.com/earth-engine/guides/auth).


In [ ]:
# Attempt to initialize. If it fails, authenticate and ask for project ID if needed.
try:
    ee.Initialize()
except Exception:
    print("Authentication required...")
    ee.Authenticate()
    try:
        ee.Initialize()
    except ee.EEException:
        print("No default project found.")
        project_id = input("Please enter your Google Cloud Project ID: ")
        ee.Initialize(project=project_id)

## **3. Definition of the remote GLANCE dataset source.**

In [ ]:
# 1. Setup automatic reloading of external modules
%load_ext autoreload
%autoreload 2

# 2. Import the custom function from the utils file
from utils import get_glance_map

# 3. Define parameters
target_year = 2015

# 4. Run the function imported from utils.py
glance_map = get_glance_map(target_year)

# 5. Display the map
if glance_map:
    display(glance_map)

GLANCE map for year 2015 generated successfully.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', trans…